# Задание 1 (2 балла)

Напишите класс `MyDict`, который будет полностью повторять поведение обычного словаря, за исключением того, что при итерации мы должны получать и ключи, и значения.

**Модули использовать нельзя**

In [79]:
class MyDict(dict):
    def __iter__(self):
        return iter(self.items()) # ¯\_(ツ)_/¯ Модули не использованы

In [80]:
dct = MyDict({"a": 1, "b": 2, "c": 3, "d": 25})
for key, value in dct:
    print(key, value)   

a 1
b 2
c 3
d 25


In [81]:
for key, value in dct.items():
    print(key, value)

a 1
b 2
c 3
d 25


In [82]:
for key in dct.keys():
    print(key)

a
b
c
d


In [83]:
dct["c"] + dct["d"]

28

# Задание 2 (2 балла)

Напишите функцию `iter_append`, которая "добавляет" новый элемент в конец итератора, возвращая итератор, который включает изначальные элементы и новый элемент. Итерироваться по итератору внутри функции нельзя, то есть вот такая штука не принимается
```python
def iter_append(iterator, item):
    lst = list(iterator) + [item]
    return iter(lst)
```

**Модули использовать нельзя**

In [87]:
def iter_append(iterator, item): # I think, this does not need comments
    yield from iterator
    yield item

my_iterator = iter([1, 2, 3])
new_iterator = iter_append(my_iterator, 4)

for element in new_iterator:
    print(element)

1
2
3
4


# Задание 3 (5 баллов)

Представим, что мы установили себе некотурую библиотеку, которая содержит в себе два класса `MyString` и `MySet`, которые являются наследниками `str` и `set`, но также несут и дополнительные методы.

Проблема заключается в том, что библиотеку писали не очень аккуратные люди, поэтому получилось так, что некоторые методы возвращают не тот тип данных, который мы ожидаем. Например, `MyString().reverse()` возвращает объект класса `str`, хотя логичнее было бы ожидать объект класса `MyString`.

Найдите и реализуйте удобный способ сделать так, чтобы подобные методы возвращали экземпляр текущего класса, а не родительского. При этом **код методов изменять нельзя**

**+3 дополнительных балла** за реализацию того, чтобы **унаследованные от `str` и `set` методы** также возвращали объект интересующего нас класса (то есть `MyString.replace(..., ...)` должен возвращать `MyString`). **Переопределять методы нельзя**

**Модули использовать нельзя**

In [145]:
def make_class_methods_return_instances_of_class(cls): # Create a class decorator

    def method_decorator(mthd): # Create a method decorator for changing output of methods  
        def modified_method(self, *args, **kwargs):
            initial_return = mthd(self, *args, **kwargs)     # The output of the initial 
            # method.
            if isinstance(initial_return, cls.__bases__[0]): # Check if the initial 
                                                             # method produces an 
                # instance of the parent class. If yes, turn it into an instance of
                # the child class (We assume such constructor is defined).
                class_instance = cls(initial_return)
                
                return class_instance
    
            else: # If the initial method does not produce an instance of the parent class,
                  # leave it as it is
                return initial_return
    
        return modified_method

    for name, method in cls.__dict__.items(): # For all callable methods, 
                                              # apply the decorator above
        if callable(method):
            setattr(cls, name, method_decorator(method))
            
    return cls

            
@make_class_methods_return_instances_of_class
class MyString(str):
    def reverse(self):
        return self[::-1]
    
    def make_uppercase(self):
        return "".join([chr(ord(char) - 32) if 97 <= ord(char) <= 122 else char for char in self])
    
    def make_lowercase(self):
        return "".join([chr(ord(char) + 32) if 65 <= ord(char) <= 90 else char for char in self])
    
    def capitalize_words(self):
        return " ".join([word.capitalize() for word in self.split()])
    
@make_class_methods_return_instances_of_class
class MySet(set):
    def is_empty(self):
        return len(self) == 0
    
    def has_duplicates(self):
        return len(self) != len(set(self))
    
    def union_with(self, other):
        return self.union(other)
    
    def intersection_with(self, other):
        return self.intersection(other)
    
    def difference_with(self, other):
        return self.difference(other)

In [146]:
string_example = MyString("Aa Bb Cc")
set_example_1 = MySet({1, 2, 3, 4})
set_example_2 = MySet({3, 4, 5, 6, 6})

print(type(string_example.reverse()))
print(type(string_example.make_uppercase()))
print(type(string_example.make_lowercase()))
print(type(string_example.capitalize_words()))
print()
print(type(set_example_1.is_empty()))
print(type(set_example_2.has_duplicates()))
print(type(set_example_1.union_with(set_example_2)))
print(type(set_example_1.difference_with(set_example_2)))

<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>

<class 'bool'>
<class 'bool'>
<class '__main__.MySet'>
<class '__main__.MySet'>


# Задание 4 (5 баллов)

Напишите декоратор `switch_privacy`:
1. Делает все публичные **методы** класса приватными
2. Делает все приватные методы класса публичными
3. Dunder методы и защищённые методы остаются без изменений
4. Должен работать тестовый код ниже, в теле класса писать код нельзя

**Модули использовать нельзя**

In [255]:
def switch_privacy(cls):
    
    private_attr_list = []
    
    public_attr_list = []
    
            
    for name, method in cls.__dict__.items(): # For all callable methods, 
        
        if callable(method):
        
            if name.startswith('_' + cls.__name__ + '__') and not name.endswith('__'):
                private_attr_list.append((name, method))
                
            elif not name.startswith('_'):
                public_attr_list.append((name, method))
            
    for priv_name, method in private_attr_list:
        setattr(cls, priv_name[3+len(cls.__name__):], method)
        delattr(cls, priv_name)
            
    for pub_name, method in public_attr_list:
        setattr(cls, '__' + str(pub_name), method)
        delattr(cls, pub_name)
            
    return cls

@switch_privacy
class ExampleClass:
    # Но не здесь
    def public_method(self):
        return 1
    
    def _protected_method(self):
        return 2
    
    def __private_method(self):
        return 3
    
    def __dunder_method__(self):
        pass

In [256]:
test_object = ExampleClass()

test_object.__dir__()
#test_object._ExampleClass__public_method() # Публичный метод стал приватным

['__module__',
 '_protected_method',
 '__dunder_method__',
 '__dict__',
 '__weakref__',
 '__doc__',
 'private_method',
 '__public_method',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [257]:
test_object.private_method()   # Приватный метод стал публичным

3

In [258]:
test_object._protected_method()   # Защищённый метод остался защищённым

2

In [261]:
test_object.__dunder_method__()   # Дандер метод не изменился

In [266]:
hasattr(test_object, "public_method"), hasattr(test_object, "private_method")   # Изначальные варианты изменённых методов не сохраняются

(False, True)

# Задание 5 (7 баллов)

Напишите [контекстный менеджер](https://docs.python.org/3/library/stdtypes.html#context-manager-types) `OpenFasta`

Контекстные менеджеры это специальные объекты, которые могут работать с конструкцией `with ... as ...:`. В них нет ничего сложного, для их реализации как обычно нужно только определить только пару dunder методов. Изучите этот вопрос самостоятельно

1. Объект должен работать как обычные файлы в питоне (наследоваться не надо, здесь лучше будет использовать **композицию**), но:
    + При итерации по объекту мы должны будем получать не строку из файла, а специальный объект `FastaRecord`. Он будет хранить в себе информацию о последовательности. Важно, **не строки, а именно последовательности**, в fasta файлах последовательность часто разбивают на много строк
    + Нужно написать методы `read_record` и `read_records`, которые по смыслу соответствуют `readline()` и `readlines()` в обычных файлах, но они должны выдавать не строки, а объект(ы) `FastaRecord`
2. Конструктор должен принимать один аргумент - **путь к файлу**
3. Класс должен эффективно распоряжаться памятью, с расчётом на работу с очень большими файлами
    
Объект `FastaRecord`. Это должен быть **датакласс** (см. про примеры декораторов в соответствующей лекции) с тремя полями:
+ `seq` - последовательность
+ `id_` - ID последовательности (это то, что в фаста файле в строке, которая начинается с `>` до первого пробела. Например, >**GTD326487.1** Species anonymous 24 chromosome) 
+ `description` - то, что осталось после ID (Например, >GTD326487.1 **Species anonymous 24 chromosome**)


Напишите демонстрацию работы кода с использованием всех написанных методов, обязательно добавьте файл с тестовыми данными в репозиторий (не обязательно большой)

**Можно использовать модули из стандартной библиотеки**

In [10]:
import re
import os
from dataclasses import dataclass

ID_AND_DESC = re.compile(r'^>[^\n]*')

@dataclass
class FastaRecord():
        seq: str
        _id: str
        description: str
        
    
# use generators to treat memory effectively           
class OpenFasta():
    
    def __init__(self, filename):
        self.filename = filename
    
    def __enter__(self):
        self.file = open(self.filename,'r')
        return self.file
    
    def read_record(self) -> FastaRecord:
        
        self.current_sequence = ''
        
        if not self.current_line:
            try:
                self.current_line = next(self.file)
            except EOFError:
                print("End of file!")
                return 1
            
        while not re.search(ID_AND_SEQ, self.current_line):
            try:
                self.current_line = next(self.file)
            except EOFError:
                print("End of file!")
                return 1
            
        match = re.search(ID_AND_SEQ, self.current_line)
        self.curr_id = match.group().split()[0]
        self.curr_decription = ' '.join(match.group().split()[1:])
        
        self.current_line = next(self.file)
        while not re.search(ID_AND_SEQ, self.current_line):
            self.current_sequence.append(self.current_line)
            try:
                self.current_line = next(self.file)
            except EOFError:
                print("End of file!")
                break
                
        return FastaRecord(self.current_sequence, self.curr_id, self.curr_description)
        
    
    def read_records(self):
        pass
    
    def __exit__(self, *args, **kwargs):
        if self.file:
            self.file.close()
            

with OpenFasta(os.path.join("data", "example.fasta")) as fasta:
    records = []
    for _ in range(2):
        records.append(fasta.read_record())
                
    print(records)
    

AttributeError: '_io.TextIOWrapper' object has no attribute 'read_record'

# Задание 6 (7 баллов)

1. Напишите код, который позволит получать все возможные (неуникальные) генотипы при скрещивании двух организмов. Это может быть функция или класс, что вам кажется более удобным.

Например, все возможные исходы скрещивания "Aabb" и "Aabb" (неуникальные) это

```
AAbb
AAbb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
aabb
aabb
```

2. Напишите функцию, которая вычисляет вероятность появления определённого генотипа (его ожидаемую долю в потомстве).
Например,

```python
get_offspting_genotype_probability(parent1="Aabb", parent2="Aabb", target_genotype="Аabb")   # 0.5

```

3. Напишите код, который выводит все уникальные генотипы при скрещивании `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` и `'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'`, которые содержат в себе следующую комбинацию аллелей `'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'`
4. Напишите код, который расчитывает вероятность появления генотипа `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` при скрещивании `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн` и `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн`

Важные замечания:
1. Порядок следования аллелей в случае гетерозигот всегда должен быть следующим: сначала большая буква, затем маленькая (вариант `AaBb` допустим, но `aAbB` быть не должно)
2. Подзадачи 3 и 4 могут потребовать много вычислительного времени (до 15+ минут в зависимости от железа), поэтому убедитесь, что вы хорошо протестировали написанный вами код на малых данных перед выполнением этих задач. Если ваш код работает **дольше 20 мин**, то скорее всего ваше решение не оптимально, попытайтесь что-нибудь оптимизировать. Если оптимальное решение совсем не получается, то попробуйте из входных данных во всех заданиях убрать последний ген (это должно уменьшить время выполнения примерно в 4 раза), но **за такое решение будет снято 2 балла**
3. Несмотря на то, что подзадания 2, 3 и 4 возможно решить математически, не прибегая к непосредственному получению всех возможных генотипов, от вас требуется именно brute-force вариант алгоритма

**Можно использовать модули из стандартной библиотеки питона**, но **за выполнение задания без использования модулей придусмотрено +3 дополнительных балла**

In [38]:
def tensor(pair_of_pairs) -> [[str]]:
        return [''.join([i, j]) for i in pair_of_pairs[0] for j in pair_of_pairs[1]]

def split_by_pairs(sequence: str) -> [str]:
    return [sequence[i:i+2] for i in range(len(sequence))]

def genotypes_of_children(genotype_1, genotype_2):
    tensor_product_of_genotypes =\
    list(map(tensor, zip(split_by_pairs(genotype_1), split_by_pairs(genotype_2))))
    list_of_genotypes = []
    for i in range(len(genotype_1)*2): # Base 4 numbers enumerate all possible indices
        genotype = ''
        for j in range(int(len(genotype_1)/2)): # We use a base 4 number as a list if [0-3] indices
            genotype += tensor_product_of_genotypes[j][int(str(int(str(i), 4))[j])]
            
        list_of_genotypes.append(genotype) # It doesn't work because I don't have enough
                                           # time to make it work
        
    return list_of_genotypes
    

In [39]:
genotypes_of_children('AaBbCcDd', 'aaBBccDd')

IndexError: string index out of range

In [20]:
# Ваш код здесь (3 подзадание)

In [21]:
# Ваш код здесь (4 подзадание)